In [4]:
import sys
sys.path.append("../src")

import pandas as pd

import plotly.graph_objs as go
import plotly.offline as py

import utilities as u
import constants as c

In [11]:
df_l = pd.read_excel("../data/data.xlsx", "liquid_m", index_col=0)
# Add total column
if c.names.TOTAL in df_l.columns:
    del df_l[c.names.TOTAL]

df_l[c.names.TOTAL] = df_l.sum(axis=1)

df_t = pd.read_excel("../data/data.xlsx", "trans_m", index_col=0)
df_t = u.dfs.fix_df_trans(df_t)

In [12]:
df_t = u.dfs.group_df_by(df_t[df_t[c.cols.TYPE] == c.names.EXPENSES], "M")
df_t = df_t.rolling(12, min_periods=1).mean()

In [13]:
df_l[c.names.TOTAL]/df_t[c.cols.AMOUNT]

Date
2011-12-01    10.531464
2012-01-01     9.700204
2012-02-01     9.442228
2012-03-01     9.436206
2012-04-01     9.015481
2012-05-01    10.194742
2012-06-01    10.224493
2012-07-01     8.432094
2012-08-01     7.243886
2012-09-01     7.346356
2012-10-01     6.616522
2012-11-01     6.746330
2012-12-01     6.708467
2013-01-01     7.107595
2013-02-01     6.722949
2013-03-01     6.933032
2013-04-01     7.739603
2013-05-01     9.075986
2013-06-01    10.183835
2013-07-01    13.873570
2013-08-01    11.678960
2013-09-01     9.332456
2013-10-01     7.994978
2013-11-01     7.808351
2013-12-01     7.910961
2014-01-01     7.812598
2014-02-01     8.095542
2014-03-01     8.618638
2014-04-01     9.159604
2014-05-01     9.797465
                ...    
2016-02-01     8.929011
2016-03-01    12.744775
2016-04-01     8.927131
2016-05-01    13.309230
2016-06-01    13.922927
2016-07-01    20.036929
2016-08-01    19.799660
2016-09-01    13.729843
2016-10-01    10.888426
2016-11-01    10.632733
2016-12-01 